In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import duckdb
import vectorbt as vbt

from helper import calculate_triple_barrier_labels
from helper import calculate_rolling_std

In [18]:
def QUERY(query):
    # Connect to DuckDB
    with duckdb.connect(
        database = '/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/data/database.db',
        read_only = False
    ) as conn:
        # Execute the query
        result = conn.sql(query).df().set_index('time_period_end').dropna().asfreq('1min', method = 'ffill')
        result = result.drop(columns = ['time_period_start', 'time_open', 'time_close', 'trades_count'], axis = 1)
        # Return the result
        return result

def construct_dataset_for_ml():
    with duckdb.connect(
        database = '/Users/louisspencer/Desktop/Trading-Bot-Data-Pipelines/data/database.db',
        read_only = False
    ) as conn:
        # Get all distinct assets in the database
        assets = conn.sql("SELECT DISTINCT asset_id_base, asset_id_quote, exchange_id FROM market_data.price_data_1m").df()

        # Create an empty DataFrame to store the final dataset
        dataset = pd.DataFrame()

        # Loop through each asset
        for i in range(len(assets)):
            print(f"Processing asset {i + 1} of {len(assets)} ({assets.iloc[i]['asset_id_base']}/{assets.iloc[i]['asset_id_quote']} on {assets.iloc[i]['exchange_id']})...")
            # Get the asset
            asset = assets.iloc[i]

            # Get the asset data
            data = conn.sql(
                f"""
                SELECT *
                FROM market_data.price_data_1m
                WHERE
                    asset_id_base = '{asset['asset_id_base']}' AND
                    asset_id_quote = '{asset['asset_id_quote']}' AND
                    exchange_id = '{asset['exchange_id']}'
                """
            ).df()

            # Calculate the triple barrier labels
            data['triple_barrier_label'] = calculate_triple_barrier_labels(data, window = 60 * 24 * 7, max_holding_time = 60 * 24)

            # Add the asset data to the dataset
            dataset = pd.concat([dataset, data])

        # Return the dataset
        return dataset
    
qnt_usdt_binance = QUERY(
"""
SELECT *
FROM market_data.price_data_1m
WHERE
    asset_id_base = 'QNT' AND
    asset_id_quote = 'USDT' AND
    exchange_id = 'BINANCE'
"""
)
qnt_usdt_binance

,price_open,price_high,price_low,price_close,volume_traded,asset_id_base,asset_id_quote,exchange_id
time_period_end,,,,,,,,
2021-07-28 23:02:00-07:00,135.1,150.0,135.1,149.3,2421.519,QNT,USDT,BINANCE
2021-07-28 23:03:00-07:00,149.3,156.7,146.1,147.8,5550.838,QNT,USDT,BINANCE
2021-07-28 23:04:00-07:00,147.8,148.7,130.6,135.2,3699.252,QNT,USDT,BINANCE
2021-07-28 23:05:00-07:00,135.1,135.2,126.0,130.8,1815.616,QNT,USDT,BINANCE
2021-07-28 23:06:00-07:00,131.3,134.1,128.8,132.2,1944.813,QNT,USDT,BINANCE
...,...,...,...,...,...,...,...,...
2024-05-28 12:48:00-07:00,91.3,91.3,91.2,91.3,85.663,QNT,USDT,BINANCE
2024-05-28 12:49:00-07:00,91.4,91.4,91.3,91.3,6.266,QNT,USDT,BINANCE
2024-05-28 12:50:00-07:00,91.3,91.3,91.3,91.3,0.374,QNT,USDT,BINANCE


In [16]:
df = construct_dataset_for_ml()
df

Processing asset 1 of 70 (MANA/USDT on OKEX)...
Processing asset 2 of 70 (CVX/USDT on KUCOIN)...
Processing asset 3 of 70 (MAGIC/USDT on KUCOIN)...
Processing asset 4 of 70 (ANKR/USD on COINBASE)...
Processing asset 5 of 70 (DYDX/USDT on BINANCE)...
Processing asset 6 of 70 (AAVE/USDT on KUCOIN)...
Processing asset 7 of 70 (RUNE/USDT on POLONIEX)...


IndexError: index 10079 is out of bounds for axis 0 with size 370

In [4]:
df.triple_barrier_label.value_counts(normalize=True)

triple_barrier_label
 1.0    0.522278
-1.0    0.477722
Name: proportion, dtype: float64